In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct
model_path = "../data11/model/LLM/EXAONE-3.0-7.8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/flash_attn/ops/triton/layer_norm.py:959: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/flash_attn/ops/triton/layer_norm.py:1018: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout, *args):


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
'''
# 모델 저장
save_directory = "../data11/model/LLM/EXAONE-3.0-7.8B-Instruct"  # Specify your directory

# Save the model and tokenizer to a specified directory
if not os.path.exists(save_directory):
    os.makedirs(save_directory)
    
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
'''

In [2]:
def parse_sllm_output(sllm_output):
    # Split the output into lines
    lines = sllm_output.split("\n")
    
    # Initialize variables for question and answer
    question = ""
    answer = ""
    is_answer = False
    
    # Parse the lines to extract Q and A
    for line in lines:
        if line.startswith("[|user|]"):
            question = line.replace("[|user|]", "").strip()
        elif line.startswith("[|assistant|]"):
            is_answer = True
            answer = line.replace("[|assistant|]", "").strip()
        elif is_answer:
            answer += "\n" + line.strip()
    
    # Remove [|endofturn|] from the answer if it exists
    answer = answer.replace("[|endofturn|]", "").strip()
    
    # Return the parsed Q and A
    return {answer}
    #return f"Q: {question}\nA: {answer}"

In [3]:
context = '''
[동해종합기술][원격지원][20200219] ARcBridge2013 관련 3가지 문제

Q:
- ARcBridge2013 에서 구조계산서 출력할때, “C:\Program Files (x86)\한길아이티\ARcBridge2013\\_tmpw0.wmf 파일이 존재하지 않거나 사용중입니다.” 에러 발생

A:
- 차단 로그 없음
- EZis-C 완전종료 해도 마찬가지
- 프로그램 개발사 쪽에 문의해본다고 함.

Q:
- ARcBridge2013 SAP2000 출력 누르면, “Unable Start .xxxxx” 에러가 뜬다.

A:
- 차단 로그 없음
- EZis-C 완전종료 해도 마찬가지
- 프로그램 개발사 쪽에 문의해본다고 함.

Q:
- AEdit.exe 에서 파일 열기 시, 로컬보안드라이브가 안보인다.

A:
- 보안프로그램으로 등록해줌.


[삼성경제연구소][전화통화][20200527] 나스카 DRM (NSCHIM.EXE) 보안 프로그램 등록

Q:
보안위반내역에 나스카 DRM (NSCHIM.EXE) 의 차단 로그가 많이 쌓인다. 보안 프로그램으로 등록해도 될지.. 지난번에 NSCHILL.EXE 는 등록했는데..

A:
등록해도 된다. DRM 프로그램중 유출될 일이 없는 UI 없는 프로그램은 보안 프로그램(일반드라이브 차단 안함)으로 해도 된다.


[모코엠시스][받은메일][20201012] [D2엔지니어링] 이슈사항 공유

Q:
IRONCAD, ZWCAD 가 제대로 동작 하지 않습니다.
- 보안프로그램 등록은 하였습니다.
- 로컬 보안디스크에는 저장되지만 M드라이브에는 저장되지 않았습니다.
이 부분은 SHARE_FILE_INFO의 LFILE_EXT가 32자를 넘어서 발생한 오류로 보여, VARCHAR 64로 변경 후 테스트 예정입니다.
- M드라이브에서 CAD파일 열 때 파일이 열리지 않고 행이 발생(ZWCAD)
- CAD파일 용량은 20MB이하 파일로 테스트 하였습니다.

A:
- ZWCAD 관련하여, 로컬보안드라이브에 ZWCAD 파일은 열리는데, 엠드라이브 똑같은 파일은 열면, 행이 걸린다.
- 차단로그에는 ZwUpdHost.exe 파일이 차단된다. ZwUpdHost.exe 를 보안프로그램으로 등록해주어야 될듯 싶다.
- 엠드라이브 ZWCAD 파일 열기시 행 걸리는 문제는 위 보안프로그램등록과는 별개로 파악이 필요함.
'''

query = '''
DRM 보안프로그램으로 등록해도 되나요?
'''

prompt = f'''
다음 검색된 내용을 가지고 질문에 답하십시오.
답을 모르면 모른다고만 하십시오.
최대 3개의 문장을 사용하고 간결하게 답변하세요.
내용:{context}
Q: {query}
A:
'''

In [ ]:
prompt

In [4]:
import time

start_time = time.time()  # 시작 시간 기록

# Choose your prompt
#prompt = "Explain who you are"  # English example
#prompt = "대한민국에 모코엠시스 it 회사는 어떤 회사인가요?"   # Korean example

# Add temperature

messages = [
    {"role": "system", 
     "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

output = model.generate(
    input_ids.to("cuda"),
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=1024,
    temperature=1  
)

#print(tokenizer.decode(output[0]))

# Parse the output
parsed_output = parse_sllm_output(tokenizer.decode(output[0]))

end_time = time.time()  # 종료 시간 기록
execution_time = end_time - start_time  # 실행 시간 계산
    
# Print the result
print(f'*시간 : {execution_time}')
print(parsed_output)


*시간 : 2.5853683948516846
{'등록해도 됩니다. DRM 프로그램 중 유출될 일이 없는 UI 없는 프로그램은 보안 프로그램으로 등록해도 안전합니다. 단, 차단 로그가 많이 쌓이는 경우 해당 프로그램을 보안 프로그램으로 등록해도 문제가 해결되지 않을 수 있으니 추가 확인이 필요합니다.'}
